<table align="left" style="border-style: hidden" class="table"> <tr><td class="col-md-2"><img style="float" src="../icon.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2023</h4><p>Ani Adhikari</p>CC BY-NC-SA 4.0</div></td></tr></table><!-- not in pdf -->

This content is protected and may not be shared, uploaded, or distributed.

In [ ]:
# SETUP
from datascience import *
from prob140 import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pylab
from scipy import stats
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

In [ ]:
def search(x_limits, cdf, u):
    """
    Runs a binary search to find the inverse cdf.
    """
    # Handle possible asymptotes.
    if cdf(x_limits[0]) > u:
        return x_limits[0]
    if cdf(x_limits[1]) < u:
        return x_limits[1]
    
    mid = (x_limits[0] + x_limits[1])/2
    diff = u - cdf(mid)
    if np.abs(diff) < 0.01:
        return mid
    if diff < 0:
        return search((x_limits[0], mid), cdf, u)
    return search((mid, x_limits[1]), cdf, u)

def plot_axes(cdf_table):
    values = cdf_table.column(cdf_table.num_columns - 1)
    cum = list(np.cumsum(values))
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    plt.yticks([0] + cum)
    plt.ylim(-0.1, 1.1)
    plt.plot([0,0], [0,1], color="k", lw=3)
    plt.xlim(-0.02, 1)
    plt.scatter([0]*(len(cum) + 1),
                [0] + cum, s=55, color="k")

def plot_discrete_cdf(cdf_table, u=None):
    """
    Plots the cdf of a discrete distribution.
    
    Parameters
    ----------
    cdf_table : Table
        Table of cdf values.
    u : float
        Value from (0, 1) to plot inverse cdf of.
    """
    values = cdf_table.column(0)
    values = np.append(values[0] - 2, values)

    cum = cdf_table.column(cdf_table.num_columns - 1)
    cum = np.append(0, np.cumsum(cum))

    for i in range(len(values) - 1):
        plt.plot([values[i], values[i+1]], [cum[i], cum[i]],
                 color="darkblue")
        plt.plot([values[i+1], values[i+1]], [cum[i], cum[i+1]],
                 ls="--", color="darkblue" )
    plt.scatter(values, cum, s=50, color="darkblue")    

    plt.plot([values[-1], values[-1] + 2], [1,1],
             color="darkblue")

    plt.xlim(values[0], values[-1] + 2)
    plt.ylim(-0.1, 1.1)
    plt.xlabel('$x$')
    plt.ylabel('CDF at $x$')
    plt.title('Graph of CDF');
    
    if u != None:
        for i in range(len(values)):
            if u <= cum[i]:
                index = values[i]
                break
        height = u
        
        plt.plot([values[0], (index+values[0])/2], [height, height],
                 marker='>', color='red', lw=1)
        plt.plot([(index+values[0])/2, index], [height, height],
                 color='red', lw=1)
        plt.plot([index, index], [height, height/2], marker="v",
                 color="red", lw=1)
        plt.plot([index, index], [0, height/2], color="red", lw=1)

def plot_continuous_cdf(x_limits, cdf, u=None):
    """
    Plots the cdf of a continuous distribution.
    """
    x = np.linspace(*x_limits, 100)
    cdf_values = list(map(cdf, x))
    plt.plot(x, cdf_values, color="darkblue")
    plt.xlabel('$x$')
    plt.ylabel('CDF at $x$')
    plt.title('Graph of CDF');
    
    if not u is None:
        index = search(x_limits, cdf, u)
        height = u

        plt.plot([x_limits[0], (index+x_limits[0])/2],
                 [height, height], marker='>', color='red', lw=1)
        plt.plot([(index+x_limits[0])/2, index],
                 [height, height], color='red', lw=1)
        plt.plot([index, index], [height, height/2],
                 marker="v", color="red", lw=1)
        plt.plot([index, index], [0, height/2], color="red", lw=1)


    plt.xlim(*x_limits)

def unit_interval_to_discrete(cdf_table):
    uniform_slider = widgets.FloatSlider(
        value=0.5,min=0,max=1,step=0.02, description='u')
    @interact(u = uniform_slider)
    def plot(u):
        plot_discrete_cdf(cdf_table, u)


def unit_interval_to_continuous(x_limits, cdf):
    uniform_slider2 = widgets.FloatSlider(
        value=0.5, min=0,max=1,step=0.02, description='u')
    
    @interact(u = uniform_slider2)
    def plot(u):
        if (cdf(u) > x_limits[1] or cdf(u) < x_limits[0]):
            plot_continuous_cdf(x_limits, cdf)
        else:
            plot_continuous_cdf(x_limits, cdf, u)
    
def override_hist(*args, **kwargs):
    """
    This cleans up some unfortunate floating point precision
    bugs in the datascience library
    """
    #kwargs['edgecolor'] = 'w'
    Table.hist2(*args, **kwargs)
    ax = plt.gca()
    ticks = ax.get_xticks()
    if np.any(np.array(ticks) != np.rint(ticks)):
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

if not hasattr(Table, 'hist2'):
    Table.hist2 = Table.hist
    
Table.hist = override_hist

# Lab 5: Densities and Transformation #

Calculus helps us work with random variables that have a continuum of possible values, such as the whole real line, or all positive numbers, or the unit interval $(0, 1)$. In this lab you will learn how to use Python to do the calculus and implement the theory that governs such random variables.

In the process, you will discover how computational systems such as `SciPy` simulate random variables with a specified distribution. Whenever we have said, "Let the number of trials have the Poisson distribution," we have tacitly assumed that we know how to generate a Poisson random number. But we never discussed how to do that, until now. The method that you will develop starts with simulating discrete random variables and then extends to the continuous case.

What you will learn in this lab:

- How to use `SymPy`, the symbolic math library of Python
- How to implement the theory of probability densities
- How to transform a uniform random variable to simulate a random variable with a specified distribution
- How to implement the change of variable formula for the density of a transformed variable

In Part A of the lab, you will set up the basics: how to use `SymPy` to do calculus, how to work with densities, and how to simulate discrete random variables.

In Part B, you will figure out how computational systems simulate continuous random variables, and how to find the densities of transformed random variables.

\newpage

# Part A of the lab starts here. # 

## Identify Your Lab Partner ##

This is a multiple choice question. Please select **ONE** of following options that best describes how you complete this lab.

- I am doing this lab by myself and I don't have a partner.
- My partner for this lab is [PARTNER'S NAME] with email [berkeley.edu email address]. [SUBMITTER'S NAME] will submit to Gradescope and add the other partner to the group on Gradescope after submission.

Please copy and paste **ONE** of above statements and fill in blanks if needed. If you work with a partner, make sure only one of you submit on Gradescope and that the other member of the group is added to the submission on Gradescope. Refer to the bottom of the notebook for submission instructions.


**Type your answer in this cell.**

\newpage

## Section 1: Operations in `SymPy`

When you work with densities, there is often some calculus involved. Sometimes, recognizing integrals as probabilistic quantities (probabilities, expectations, and so on) can help reduce the calculus. 

But sometimes you just have to crank out the calculus. This can occasionally be messy. Fortunately, Python can help.

The `SymPy` library is a set of tools that help you do symbolic math on the computer. To see what this means, let's start with a simple numerical calculation of a familiar sort.

In [ ]:
x = 3
(2 * x + 1) ** 2

What you can do in `SymPy`, assuming that you have explained to `SymPy` that $x$ is a symbol, is to have `(2*x + 1) ** 2` appear as $(2x + 1)^2$. You can then expand it to get $4x^2 + 4x + 1$. `SymPy` does the math that you did some time ago in algebra class, and renders the math in symbols. It gives you an exact symbolic answer, instead of numerical answer (that is often an approximation) in a particular case.

`SymPy` contains a large number of mathematical functions and operations, including integration and differentiation. This can be very handy when working with densities.

#### Getting Started ###
We must import the library and add a line of code that makes the math appear in the way that it appears in the [textbook](http://prob140.org/textbook/content/README.html).

In [ ]:
from sympy import *
init_printing()

We will start with simple examples of the basic operations and syntax. For more, take a look at the [SymPy Tutorial](https://docs.sympy.org/latest/tutorial/index.html).

### 1a) Creating a Symbolic Variable ###

You are used to assigning names to numbers, arrays, and so on. In `SymPy`, we create a symbol using `Symbol`. Its required argument is a string that contains the symbol we want to use. We can assign that to a name, just as we have done before.

In [ ]:
# Create a symbol

x = Symbol('x')
x

### 1b) Constructing a Symbolic Expression ###

Consider the math function $f(x) ~ = ~ 2x + 1$. We can create a `SymPy` expression equal to the right hand side, as follows.

In [ ]:
# An expression that is the right hand side
# of the definition of f(x)

2*x + 1

In [ ]:
# Name the expression and display it

f = 2*x + 1
f

It is important to make a distinction between the math function $f$ and the `SymPy` expression `f`. Though `f` doesn't have an $x$ in the name, it is equal to $f(x)$, the math function $f$ evaluated at the point $x$.

You can create new expressions by using math operations.

In [ ]:
f ** 2

You can ask `SymPy` to expand the square, to get an equivalent expression.

In [ ]:
expanded = expand(f ** 2)
expanded

### 1c) Evaluating an Expression at a Point ###

To evaluate $f(5)$ in math, we have take the expression $2x + 1$ and substitute the generic $x$ with the specific value $5$. So also in `SymPy`, we evaluate the symbolic expression `f` at the point $5$ by using substitution.

In [ ]:
# Evaluate the expression at the point x = 5

f.subs(x, 5)

Points themselves can be symbols. For example, `SymPy` recognizes `pi` as $\pi$. So we can evaluate the function $f$ at $\pi$:

In [ ]:
f.subs(x, pi)

You can see that `SymPy` makes choices about exactly how to display expressions. All symbolic math systems make such decisions, but not always in the same way. 

It is important to notice the difference between symbolic and numeric calculations such as those you perform using `NumPy`. For example, recall that `np.pi` is the `NumPy` expression for $\pi$, and run the cell below. 

In [ ]:
f.subs(x, pi), f.subs(x, np.pi)

The first expression is exact, as in math, while the numerical calculation results in an approximate value by computation.

### 1d) Integration ###
Integration is one of the most important operations in probability. Let's learn how to integrate functions using `SymPy`.

The indefinite integral $\int f(x)dx$ can be displayed using `Integral(f)`.

In [ ]:
# Display the indefinite integral

Integral(f)

That's nice, but it would be even nicer if `SymPy` could actually do the integral, not just show it to us.

To make this happen, we are going to be a bit rude. We are going to tell `SymPy` to just `doit()`.

In [ ]:
# Evaluate the indefinite integral

Integral(f).doit()

This creates a new `SymPy` expression, which you can assign to a name as usual.

In [ ]:
F = Integral(f).doit()
F

Definite integrals can be calculated in two ways, corresponding to the left and right hand sides of the example below.

$$
\int_3^7 f(x)dx ~ = ~ F(7) - F(3)
$$

You already know how to compute the right hand side using substitution and the expression representing the indefinite integral.

In [ ]:
# right hand side

F.subs(x, 7) - F.subs(x, 3)

To display a definite integral, `Integral` requires an additional argument that specifies the variable being integrated and the two limits of integration: 

- (variable_name, lower_limit, upper_limit)

At the moment we are working with a function of just one variable, so it seems unnecessary to provide the name of the variable. But the name will become important as soon as we start double integration.

In [ ]:
# left hand side
# Display the definite integral

Integral(f, (x, 3, 7))

To evaluate the integral, use `doit()` as before. You will get the same answer as you did earlier.

In [ ]:
# Evaluate the definite integral

Integral(f, (x, 3, 7)).doit()

The standard operations on integrals can be performed just as you would expect. At each cell below, pause and read the expression carefully. Then run the cell and examine the output just as carefully.

In [ ]:
# Display a sum of integrals

Integral(f, (x, 3, 4)) + Integral(f, (x, 4, 7))

In [ ]:
# Evaluate it

(Integral(f, (x, 3, 4)) + Integral(f, (x, 4, 7)) ).doit()

In [ ]:
# Display the integral of a new function of x

Integral((x-7)*f, (x, 2, 10))

In [ ]:
# Evaluate the integral

Integral((x-7)*f, (x, 2, 10)).doit()

You could have evaluated each of these integrals without displaying it first. But it is strongly recommended that you do display the integral first, so that you are confident that the right quantity is being calculated.

### 1e) Differentiation ###
We started with $f(x) = 2x+1$:

In [ ]:
f

The derivative is
$$
\frac{d}{dx} f(x) ~ = ~ 2
$$

To get the derivative in `SymPy`, use `diff` for "differentiate".

In [ ]:
diff(f)

Since we defined $F$ as the integral of $f$, the Fundamental Theorem of Calculus says that

$$
f(x) ~ = ~ \frac{d}{dx} F(x)
$$

In [ ]:
# Differentiate F to get back f

diff(F)

In [ ]:
# You can differentiate F twice, just for fun
# That's the same as differentiating f

diff(diff(F))

### 1f) Solving an Equation ###
Equations can have numerous different forms, so symbolic math systems require consistent ways of specifying the equations to be solved. That way they don't have to have separate functions for solving all the different kinds of equations. Let's see how to solve equations in `SymPy`.

We will restrict ourselves to equations involving a single unknown. For a sense of the variety of the equations that `SymPy` can solve, see the [`Solver` module reference](https://docs.sympy.org/latest/modules/solvers/solvers.html).

The general form of an equation in $x$ is 

$$
h(x) ~ = ~ g(x)
$$

for two functions $h$ and $g$. Solving this equation is the same as solving the equation

$$
h(x) - g(x) ~ = ~ 0
$$

which can be written as $d(x) = 0$ for $d = h - g$.

So every equation can be written with 0 as the right hand side. This helps simplify syntax. That is why `Solver` in `SymPy` requires 0 as the right hand side of the equation. That is, the equation should be in the form $d(x) = 0$. 

The call is `solve(d, x)` where `d` is the expression on the left hand side and involves the symbol `x`.

Solving $d(x) = 0$ means finding the value of $x$ for which the value of $d(x)$ is 0. The instruction `solve(d, x)` tells the computer to find the value of `x` for which the value of the expression `d` is 0. 

For example, to solve $5x = 15$, `SymPy` finds the value of $x$ that solves the equation $5x - 15 = 0$. 

In [ ]:
solve(5*x - 15, x)

Some equations have multiple solutions, so the output is a list of all the solutions. In the case of the equation $5x - 15 = 0$, the list contains just one element, 3.

When you want to work with the element, you can access it by specifying its index.

In [ ]:
solve(5*x - 15, x)[0]

Here are the solutions to the equation $x^2 + 5x + 6 = 0$. In an algebra class you will have learned to factor the left hand side to get $(x+2)(x+3) = 0$ and hence the two solutions $x = -3$ and $x = -2$.

In [ ]:
solve(x**2 + 5*x + 6, x)

You can use previously defined expressions as part of the expression in `solve`. For example, we previously defined `f` as the right hand side of $f(x) = 2x+1$. We can solve the equation $f(x) = 9$ as follows.

In [ ]:
solve(f - 9, x)

### 1g) Inverse ###

Finding the inverse of a function $f$ at a point $y$ means finding the value of $x$ such that $f(x) = y$. 

This just means writing $x$ in terms of $y$. The formal math move is to solve for $x$ in the equation $f(x) = y$.

Because we are going to solve the equation using `SymPy`, we will solve for $x$ in the equation $f(x) - y = 0$.

To execute this plan in `SymPy`, we will first have to declare `y` as a symbol, and then solve the equation.

In [ ]:
y = Symbol('y')
solve(f - y, x)

Does the answer make sense? Remember that $f(x) = 2x + 1$ and solve the equation the old-fashioned way.

$$
y = 2x + 1 ~ \iff ~ y - 1 = 2x ~ \iff ~ x = \frac{y-1}{2} ~ = ~ \frac{y}{2} - \frac{1}{2}
$$

It works. As before, you can access the inverse by specifying its index in the list.

In [ ]:
f_inverse = solve(f-y, x)[0]
f_inverse

\newpage

## Section 2: Working with Densities ###
Now that you have an idea of what you can do with `SymPy`, it's time for some probability theory.

In Data 140, calculus is used almost exclusively when working with densities. Later in the course it will be used for some optimization as well, but density calculations are its main use.

In this part of the lab, you will use `SymPy` to work with a very simple density. That way, you will be able to check by hand that `SymPy` is getting the right answers. In subsequent parts of the lab, the densities will get more complicated. Treat this part of the lab as a warm-up.

### 2a) A Density on the Unit Interval ###
Let $X$ have density $f_X(x) = 6x^5$ for $0 < x < 1$.

To work with this density, you have to start by creating a symbol representing $x$. Notice that `Symbol` takes an optional argument that allows you to specify that a variable is positive. This can be useful, as you will see below. 

Complete the cell by using `x` to construct an expression `f_X` that is equal to $f_X(x)$.

In [ ]:
x = Symbol('x', positive=True)
f_X = ...
f_X

Is the function $f_X$ in fact a density? 

What do you need to check to answer this? Explain in the cell below and use the cell below that for calculation.


**Your answer here**

In [ ]:
...

The `prob140` library includes a plotting function `Plot_continuous` that plots a line graph of a probability density. The arguments are a list containing the endpoints of the interval over which to draw the graph, and a probability density which can be specified by a `SymPy` expression.

Run the cell below to plot the graph of $f_X$ on the unit interval.

In [ ]:
Plot_continuous([0, 1], f_X)
plt.title('Density of $X$');

Before you go further, please review Part **1d** on Integration.

### 2b) Finding Probabilities ###

Find $P(X < 0.8)$ by integrating the density appropriately. The code in the first cell below should display the integral and in the next cell you will evaluate that integral.

In [ ]:
# Display the integral for P(X < 0.8)

Integral(...)

In [ ]:
# Evaluate P(X < 0.8)

...

In the cell below, fill in the blanks with numbers: The event $\vert X - 0.8 \vert < 0.05$ is the same as the event $X \in (\underline{~~~~~~~~~~~}, \underline{~~~~~~~~~~~})$.


**Fill in your answer beow**

First blank: 

Second blank:

Find $P(\vert X - 0.8 \vert < 0.05)$.

In [ ]:
# P(|X - 0.8| < 0.05)

...

### 2c) Finding Expectations ###

Read [Section 15.3](http://prob140.org/textbook/content/Chapter_15/03_Expectation.html) before you get started. It might help you to write out the integrals on scratch paper before writing `SymPy` code.

Find $E(X)$.

In [ ]:
# E(X)

expectation_X = ...
expectation_X

Find $SD(X)$.

In [ ]:
# SD(X)

...

Just because you can, find $E(\log(X))$. `SymPy` recognizes `log`. Use the first cell below to display the appropriate integral and use the next cell to evaluate it.

In [ ]:
# Display the integral for E(log(X))

Integral(...)

Notice that `SymPy` writes its integrand in a different order than we do in class. That's OK; it doesn't affect the integral.

In [ ]:
# Numerical value of E(log(X))

...

Why is the answer negative?


**Your answer here**

Find $E(\sin^2(X))$, first displaying the integral and then evaluating it. Use `sin` for sine.

In [ ]:
# Display the integral for E(sin_squared(X))

...

In [ ]:
value_of_integral = ...
value_of_integral

Sometimes `SymPy` can collect terms and simplify its answers. If you are good at half-angle formulae in trigonometry, you can check by hand that the simplification below is correct. But you don't need to do that for the lab.

In [ ]:
simplify(value_of_integral)

\newpage

## Section 3: Simulation and the CDF (Discrete)

In this part of the lab, you will discover the method by which computer programs simulate discrete random variables that have a specified distribution. Later in the lab, you will use the same idea to simulate random variables that have densities.

Our starting point is a distribution on just four values. Suppose $X$ has the distribution `dist_X` below. 

In [ ]:
vals_X = [-2, 1, 4, 7]
probs_X = [0.3, 0.1, 0.2,0.4]

dist_X = Table().values(vals_X).probabilities(probs_X)
dist_X

Our goal is to simulate one value of $X$. That is, we want to come up with a process that returns one of the four possible values of $X$ with the right probabilities.

### 3a) A Vertical Unit Interval ###
The graphic below shows the probabilities in `dist_X` stacked vertically. From the bottom to the top, therefore, you have the unit interval.

In [ ]:
plot_axes(dist_X)

Now imagine throwing a dart at the unit interval. That is, let $U$ be a random variable that has the uniform distribution on $(0, 1)$, and suppose you mark the value of $U$ on the unit interval shown in the graph.

Find the following probabilities and see how they are related to the distribution of $X$.

(i) $P(U \le 0.3)$

(ii) $P(0.3 < U \le 0.4)$

(iii) $P(0.4 < U \le 0.6)$

(iv) $P(0.6 < U \le 1)$


i. **Your Answer Here**

ii. **Your Answer Here**

iii. **Your Answer Here**

iv. **Your Answer Here**

### 3b) Idea for Simulating $X$ ###
Starting with a uniform $(0, 1)$ random variable $U$, propose a method of generating a value of $X$. 

Your method should take $U$ as its input and return one of the four possible values as output, in such a way that for each $i = -2, 1, 4, 7$, the chance of returning that value $i$ is $P(X = i)$.

Just describe your method in words. No formula or code is needed.


**Your Answer Here**

### 3c) Introducing the CDF
The method `plot_discrete_cdf` takes a distribution as its argument and plots a graph of the cdf.

Run the cell below to get a graph of the cdf of the random variable $X$ defined above.

In [ ]:
plot_discrete_cdf(dist_X)

Let $F_X$ be the cdf of $X$. What is the definition of $F_X(2.57)$? Does the graph show the correct value for $F_X(2.57)$?


**Your Answer Here**

### 3d) Jumps ###
At what points $x$ does the graph have a jump? For each point $x$ at which there is a jump, find the size of the jump in terms of the distribution of $X$.


**Your Answer Here**

### 3e) From the Unit Interval to Values of $X$ ###
The function `unit_interval_to_discrete` takes a distribution as its argument and displays an animation of a method that generates one value of a random variable that has the given distribution.  The method starts with a number on the unit interval.

Run the cell below. Move the slider around and see how the returned value changes **depending on the starting value** in the unit interval. How is the method that is being displayed related to the one you proposed in **3b**?

In [ ]:
unit_interval_to_discrete(dist_X)


**Your answer here**

### 3f) A Random Starting Point ###
The method `plot_discrete_cdf` that you used earlier also takes a second argument which is a number between 0 and 1. 

Complete the cell below so that the second argument is picked uniformly at random from (0, 1). To generate one uniform $(0, 1)$ random number, use `stats.uniform.rvs(0, 1, size=1)`.

In [ ]:
plot_discrete_cdf(dist_X, ...)

Run the cell a few times. How is it related to the method you proposed in **3b** for generating a value of $X$?


**Your Answer Here**

# Part A of the lab ends here, and is due by 11:59 pm Monday, March 13th. #

## Submission Instructions ##

Many assignments throughout the course will have a written portion and a code portion. Please follow the directions below to properly submit both portions.

### Written Portion ###
*  Scan all the pages into a PDF. You can use any scanner or a phone using applications such as CamScanner. Please **DO NOT** simply take pictures using your phone. 
* Please start a new page for each question. If you have already written multiple questions on the same page, you can crop the image in CamScanner or fold your page over (the old-fashioned way). This helps expedite grading.
* It is your responsibility to check that all the work on all the scanned pages is legible.
* If you used $\LaTeX$ to do the written portions, you do not need to do any scanning; you can just download the whole notebook as a PDF via LaTeX.

### Code Portion ###
* Save your notebook using `File > Save and Checkpoint`.
* Generate a PDF file using `File > Download As > PDF via LaTeX`. This might take a few seconds and will automatically download a PDF version of this notebook.
    * If you have issues, please post a follow-up on the general Lab 5A Ed thread.
    
### Submitting ###
* Combine the PDFs from the written and code portions into one PDF. [Here](https://smallpdf.com/merge-pdf) is a useful tool for doing so. 
* Submit the assignment to Lab 5A on Gradescope. 
* **Make sure to assign each page of your pdf to the correct question.**
* **It is your responsibility to verify that all of your work shows up in your final PDF submission.**

If you are having difficulties scanning, uploading, or submitting your work, please read the [Ed Thread](https://edstem.org/us/courses/35049/discussion/2398718) on this topic and post a follow-up on the general Lab 5A Ed thread.

## **We will not grade assignments which do not have pages selected for each question.** ##

\newpage

# Part B: Transforming Continuous Variables #

## Identify your lab partner ##

This is a multiple choice question. Please select **ONE** of following options that best describes how you complete Lab 3.

- I am doing Part B of this lab by myself and I don't have a partner.
- My partner for Part B of this lab is [PARTNER'S NAME] with email [berkeley.edu email address]. [SUBMITTER'S NAME] will submit to Gradescope and add the other partner to the group on Gradescope after submission.

Please copy and paste **ONE** of above statements and fill in blanks if needed. If you work with a partner, make sure only one of you submit on Gradescope and that the other member of the group is added to the submission on Gradescope. Refer to the bottom of the notebook for submission instructions.


**Type your answer in this cell.**

\newpage

## Section 4. Extension to Continuous Distributions ##
Now suppose you want to simulate a random variable that has a specified continuous distribution. Let's start with the exponential $(\lambda)$ distribution.

Let $T$ have the exponential distribution with rate $\lambda$. Let $F_T$ be the cdf of $T$. Refer to [Section 15.4.1](http://prob140.org/textbook/content/Chapter_15/04_Exponential_Distribution.html#cdf-and-survival-function) for the formula for $F_T$, and don't forget that $F_T(t) = 0$ for $t < 0$.

### 4a) Plotting the Exponential CDF ###
For a numerical example, let $T$ be a random variable that has the exponential distribution with rate $\lambda = 0.5$, or equivalently, expectation $2$. Define a function `expon_mean2_cdf` that takes a numerical argument $x$ and returns the numerical value of $F_T(x)$. Use `np.exp(y)` for $e^y$.

Make sure your function returns the correct value of $F_T(x)$ for **all** real numbers $x$.

In [ ]:
# Don't use "lambda" as that means something else in Python
lamb = 0.5

def expon_mean2_cdf(x):
    ...

The function `plot_continuous_cdf` plots the cdf of a continuous variable. The first two arguments:
- an interval (a, b) over which to draw the cdf
- the name of a function that takes a numerical input and returns the value of the cdf at that input

Run the cell below to check that your function `expon_mean2_cdf` looks good.

In [ ]:
plot_continuous_cdf((-1, 8), expon_mean2_cdf)

### 4b) Idea for Simulating an Exponential Random Variable ###

Suppose you are given one uniform $(0, 1)$ random number and are asked to simulate $T$. Based on Part 3 of the lab, propose a method for doing this by using the graph above.

You don't have to prove that the method works. There's a formal proof in the textbook. Just propose the method.


**Your Answer Here**

### 4c) Visualizing the Idea ###
The animation in the cell below is analogous to the one in Part 3. Its arguments are: 
- a plotting interval
- the name of a continuous cdf function

The output demonstrates a method for generating a number on the positive real line by starting with a value on the unit interval that forms the vertical axis. 

Run the cell and move the slider around to see how the returned value changes depending on the starting value on the vertical axis.

In [ ]:
unit_interval_to_continuous((-1, 8), expon_mean2_cdf)

The method `plot_continuous_cdf` takes an optional third argument that is a number between $0$ and $1$. 

Complete the cell below so that the third argument is a random number picked uniformly from (0, 1). Refer to **3f** for relevant code. 

In [ ]:
...

Run the cell a few times. For generating a value of $T$, how is the output related to the method you proposed in **4b**?


**Your Answer Here**

### 4d) [ON PAPER] The General Method ###
Let $F$ be any continuous increasing cdf. That is, suppose $F$ has no jumps and no flat bits. 

Suppose you are trying to create a random variable $X$ that has cdf $F$, and suppose that all you have is $F$ and a number picked uniformly on $(0, 1)$.

(i) **Fill in the blank:** Let $U$ be a uniform $(0, 1)$ random variable. To construct a random variable $X = g(U)$ so that $X$ has the cdf $F$, take $g = \underline{~~~~~~~~~~~~~}$.


(ii) **Fill in the blank:** Let $U$ be a uniform $(0, 1)$ random variable. For the function $g$ defined by

$$
g(u) ~ = ~ \underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~}, ~~~ 0 < u < 1
$$

the random variable $X = g(U)$ has the exponential $(\lambda)$ distribution.

[Note: If $F$ is a discrete cdf then the function $g$ is complicated to write out formally, so we're not asking you to do that. The practical description of the method of simulation is in Part 3 of the lab.]

### 4e) Empirical Verification
Let's visualize what your method is doing and confirm that it works.

In the cell below, we have used `SciPy` to create a table that is called `sim` for "simulation". It consists of one column called `Uniform` that contains the values of 100,000 i.i.d. uniform $(0, 1)$ random variables. In the last line, the argument `bins=25` is used to create a histogram with 25 bins of equal width. Run the cell.

In [ ]:
N = 100000
u = stats.uniform.rvs(0, 1, size=N)
sim = Table().with_columns('Uniform', u)
sim.hist('Uniform', bins=25)

Use **4d** and the values in the column `Uniform` to create an array of values that have the exponential distribution with rate $0.5$. Use `np.log(y)` for $\log(y)$.

Then augment `sim` with a column containing the new array.

**Do not** simulate new random numbers, as you will lose the connection with the values in `Uniform`.

In [ ]:
def uniform_to_exponential_mean2(u):
    return ...

exponential_mean2 = sim.apply(...)
sim = sim.with_columns('Sim. Exponential (rate 0.5)', ...)
sim

Run the cell below to confirm that your calculation is correct.

In [ ]:
sim.hist('Sim. Exponential (rate 0.5)', bins=25)
plt.xticks(np.arange(0, 21, 2));

You have just discovered what is going on "under the hood" in functions that generate exponential random numbers. There are more complicated methods that work faster in particular cases. But this one is pretty good!

### 4f) Simulating the Standard Normal ###

Now use the general method in **4d** and the `Uniform` column of the table `sim` to generate 100,000 i.i.d. standard normal variables. Augment `sim` with a column labeled `Standard Normal` containing standard normal variables created by transforming the values in `Uniform`.

Unlike the exponential, the standard normal cdf $\Phi$ must be computed numerically, and hence so must its inverse $\Phi^{-1}$. You have to identify the appropriate `SciPy` function to transform the `Uniform` column. Do not add more lines of code.

In [ ]:

standard_normals = ...
sim = sim.with_columns('Standard Normal',
                      standard_normals)
sim.hist('Standard Normal', bins=25)

At this point, go back and look through Part 4. Notice that the only time you generated random numbers was when you simulated 100,000 uniform (0, 1) values. All the other variables were deterministic transformations of the uniform random numbers.

### Note on Uniform Random Numbers ###
Since uniform $(0, 1)$ random numbers are central to all simulations, their quality is very important for the accuracy and reliability of simulations. Testing and assessing uniform random number generators is serious business, because random number generators don't really produce random numbers. They follow deterministic processes that produce results that have properties that resemble those of random numbers. That is why they are called Pseudo Random Number Generators or PRNGs. [Python uses the Mersenne Twister](https://docs.python.org/3/library/random.html), one of the most tested and reliable PRNGs. 

\newpage

## Section 5: Densities of Transformed Variables

You now know how to transform a uniform $(0, 1)$ random variable into a random variable with any specified distribution. But often, we start with some random variable $X$, not necessarily uniform, and our analysis requires us to transform the variable. 

To find the density of the transformed variable, we can sometimes use the methods of [Chapter 16](http://prob140.org/textbook/content/Chapter_16/00_Transformations.html). In this part of the lab you will examine how to do this.

Before you begin, please review Parts 1 and 2 to recall how to use `SymPy` to work with densities. 

As in Part 2, let $X$ have density $f_X(x) = 6x^5$ for $0 < x < 1$. Run the cell below to define the density function `f_X`.

In [ ]:
x = Symbol('x', positive=True)
f_X = 6 * x**5
f_X

### 5a) Change of Variable ###
Let $g$ be a monotone differentiable function and let $V = g(X)$. The change of variable formula for densities says that the density of $V$ is given by

$$
f_V(v) = \frac{f_X(x)}{\big{|} \frac{d}{dx} g(x) \big{|}} ~~~~~ \text{evaluated at } x = g^{-1}(v)
$$

See [Section 16.2.1](http://prob140.org/textbook/content/Chapter_16/02_Monotone_Functions.html#change-of-variable-formula-for-density-increasing-function) for the derivation, and notice how the derivation depends on the cdf of $X$. The cdf was also the principal function you used to create the transformations in Parts 3 and 4 of the lab.

Each time you use the change of variable formula, the main steps to find the density are:

- Find the possible values of $V = g(X)$.
- Find the inverse of $g$.
- Find the derivative of $g$.
- Divide the density of $X$ by the derivative of $g$ (if the derivative is negative, use its absolute value instead).
- Evaluate this quotient at the inverse of $g$.

Use the formula to find the density of the area of a disc that has radius $X$. That is, find the density of $V = \pi X^2$.

Start by constructing a `SymPy` expression `g` defined by $g(x) = \pi x^2$. `SymPy` recognizes `pi` as $\pi$. Remember that you already declared `x` and `f_X(x)` at the start of this part of the lab.

In [ ]:
g = ...
g

It is worth keeping in mind that you have two representations of $\pi$: 

- `pi` produces the symbol $\pi$, using `SymPy`
- `np.pi` produces a numerical approximation to $\pi$, using `NumPy` 

In [ ]:
pi, np.pi

As always, when you are specifying a distribution, start with the possible values. What are the possible values of the random area $V$? 


**Your answer here**

Now find all the elements of the right hand side of the change of variable formula, one by one.

First find the function $g^{-1}$ by using the equation $g(x) = v$ to write $x$ in terms of $v$. That is, solve for $x$ in the equation $g(x) = v$, or, equivalently, in the equation $g(x)-v = 0$. 

Review Parts **1f** and **1g** carefully before you proceed.

In [ ]:
v = Symbol('v', positive=True)
g_inverse = solve(...)
g_inverse

Notice that `SymPy` lists just one inverse – the positive one – because you specified that $x$ is positive. The other inverse is $-\frac{\sqrt{v}}{\sqrt{\pi}}$ but it is not a permitted value of $x$ because it is negative.

Run the cell below to extract the inverse from the list.

In [ ]:
g_inverse = g_inverse[0]
g_inverse

The change of variable formula has another factor: the derivative in the denominator. Since we are working only on the interval $(0, 1)$, $g$ is an increasing function. So its derivative is positive and you won't need the absolute value in the formula.

Use `SymPy` to find the derivative of $g$, so that the expression `deriv_g` is equal to $\frac{d}{dx}g(x)$. 

**Do not** find the derviative by hand and simply assign that expression to `deriv_g`. See **1e** for how to differentiate in `SymPy`.

In [ ]:
deriv_g = ...
deriv_g

Now apply the change of variable formula to find $f_V$, the density of $V$. **In the comment line, enter the possible values of $V$.**

In [ ]:
"""For v in the interval ..., the density of V at the point v is:"""

f_V = (... / ...).subs(...)
f_V

Check that the function `f_V` is a density.

In [ ]:
...

Plot the graph of the density of $V$. Recall from Part **2a** that the first argument of `Plot_continuous` is a list of two elements: the left and right ends of the interval over which to plot the graph. They must be numerical expressions, not symbolic.

In [ ]:
Plot_continuous(..., f_V)
plt.title('Density of $V$');

For comparison, run the cell below to see the graph of $f_X$ again.

In [ ]:
Plot_continuous((0, 1), f_X)
plt.title('Density of $X$');

\newpage

## Section 6: Transforming an $F$ Distribution ##
The density you started with in Section 5 was deliberately chosen to be simple, so that you could try out the calculations by hand if you wanted to check. The real value of `SymPy` is that without much difficulty you can use essentially the same code to handle more complicated density functions.

In this part of the lab you will start with one of the famous distributions of statistical inference, confusingly named the $F$ distribution. It's not named for the cdf. It is named for Sir Ronald Fisher whose contributions include what is now the standard method of testing statistical hypotheses. 

The $F$ distribution arises as the distribution of the ratio of two independent gamma random variables (apart from a constant multiplier). The ratio arises in tests of whether three or more random samples come from the same underlying distribution. You will become closely acquainted with gamma distributions later in the course. 

For the purposes of this lab, the $F$ distribution is just an ordinary distribution on the positive numbers. It has two parameters, which we will call $n$ for "numerator" and $d$ for "denominator". And its density has a rather intimidating formula.

### The $F_{n,d}$ Density ###
The gamma function of mathematics is denoted $\Gamma$ (that's the upper case Greek letter gamma) and is defined as an integral. Both its domain and range are the positive numbers. In your current Homework assignment you are examining the definition and properties of the gamma function. In this lab, you will only need one of those results:

For positive integer $n$, $\Gamma(n) = (n-1)!$.

Now for the definition of the $F$ density.

Let $n$ and $d$ be positive integers. The random variable $X$ has the $F_{n,d}$ distribution if the density of $X$ is

$$
f_X(x) ~ = ~ \frac{\Gamma\big{(}\frac{n}{2} + \frac{d}{2}\big{)}}{\Gamma\big{(}\frac{n}{2}\big{)}\Gamma\big{(}\frac{d}{2}\big{)}} \big{(} \frac{n}{d} \big{)}^{\frac{n}{2}} x^{\frac{n}{2} -1}\big{(}1 + \frac{n}{d}x \big{)}^{-\frac{n+d}{2}} ~, ~~~~~~~ x > 0
$$

That looks awful. But it isn't, really. Let's see why.

### 6a) The Constant ###
As with many densities, the part of the formula that looks most impressive is actually the least interesting as far as the shape of the density is concerned. It's the constant of integration: the numerical factor that makes the density integrate to 1. 

To evaluate the constant you will need to evaluate the gamma function numerically. The `SymPy` function `gamma` can be used for this. 

In [ ]:
gamma(4)

In [ ]:
gamma(4.5)

In [ ]:
gamma(5)

In [ ]:
r = Symbol('r', positive=True)
gamma(r)

Start by evaluating the constant in the $F_{n,d}$ density. Define a Python function `constant_F` that takes $n$ and $d$ as its arguments and returns the normalizing constant in the density above. 

In [ ]:
def constant_F(n, d):
    return ...

Work out (by hand or by mental math) the constant when $n = d = 4$, and check that your function returns the right value.

In [ ]:
constant_F(4, 4)

As another check, work out the constant when $n = 2$ and $d = 4$, and check that your function returns the right value.

In [ ]:
constant_F(2, 4)

### 6b) The $F_{6, 4}$ Density ###

As a numerical example, let $X$ have the $F_{6, 4}$ density, which we will call $f_X$. Construct a `SymPy` expression `f_X` that is equal to $f_X(x)$.

In [ ]:
x = Symbol('x', positive=True)
f_X = constant_F(...)
f_X = nsimplify(f_X) # simplifies expression above to avoid a bug from previous semesters
f_X

That's not very scary at all. It's just a ratio of two polynomials, because $n=6$ and $d=4$ are both even. 

Check that $f_X$ is a density. Here you get to use something cute: 

- As the symbol for infinity, `SymPy` uses two lower case letter o's side by side, because oo looks like $\infty$.

In [ ]:
Integral(f_X, (x, ...)).doit()

Run the cell below to plot the density.

In [ ]:
Plot_continuous([0, 10], f_X)
plt.title('$F_{6,4}$ Density');

### 6c) A Transformation and its Inverse ###
In the rest of the lab, you will find the density of a transformation of $X$. **Review Part 5** very carefully first.

Define a new random variable $V$ by applying the following function $g$ to the random variable $X$:

$$
g(x) ~ = ~ \frac{\frac{n}{d}x}{1 + \frac{n}{d}x}
$$
Then
$$
V ~ = ~ g(X) ~ = ~ \frac{\frac{n}{d}X}{1 + \frac{n}{d}X}
$$

With the help of `SymPy`, you are going to find the density of $V$.

As always, start with the possible values. What are the possible values of V?


**Your answer here**

For $n = 6$ and $d = 4$, construct an expression `g` that is equal to $g(x)$ as defined above.

In [ ]:
g = ...
g

Is the function $g$ increasing or decreasing? Explain your answer.

[It helps to write the function as 1 minus something.]


**Your answer here**

To find the density of $V = g(X)$, you will need the inverse of $g$.

The function $g$ is monotone and therefore has a unique inverse. Let $v = g(x)$. Find the function $g^{-1}$ by completing the cell below. 

In [ ]:
v = Symbol('v', positive=True)
g_inverse = ...[...]
g_inverse

Remember that for each $v$, $g^{-1}(v)$ is a value of $x$. Since $x$ is positive, the inverse you just found better not be negative. Show by algebra (without using `SymPy`) that the inverse calculated above is positive for each possible value $v$ of $V$.


**Your answer here**

### 6d) A Derivative ###

Complete the cell below so that `deriv_g` equals $\frac{d}{dx}g(x)$.

In [ ]:
deriv_g = ...
deriv_g

Explain why this derivative is positive for all positive $x$.


**Your answer here**

### 6e) The Density of the Transformation ###
Use the change of variable formula in Part 5 to find $f_V$, the density of $V$. Start by filling in the possible values of $V$ in the comment line.

In [ ]:
# Density of V:

"""For v in the interval ..., the density of V at the point v is:"""

f_V = ...
f_V

Yikes! That looks like a horrible mess. 

Does it simplify at all? Run the next cell and see.

In [ ]:
f_V = simplify(f_V)
f_V

Algebra is truly wonderful. Especially when `SymPy` does it for you.

Recognize the density of $V$ as one of the famous ones and state its name and parameters. If you are stuck, you may find [this section](http://prob140.org/textbook/content/Chapter_17/04_Beta_Densities_with_Integer_Parameters.html#beta-densities) of the textbook helpful.


**Your answer here**

Plot the density of $V$ over all the possible values of $V$.

In [ ]:
Plot_continuous(...)
plt.title('Density of $V$');

**Note:** There is nothing special about the choices $n=6$ and $d=4$ as far as this result is concerned. If you had started out with a different $(n, d)$ pair, you would have ended up with a density in the same family but with different parameters. Based on your answer in the case $(6, 4)$, you might be able to guess what the parameters would be in general.

## Conclusion ##
Congratulations! The work that you have done in this lab will help reinforce your understanding of some of the most fundamental concepts of probability theory.

What you have learned:

- How to do symbolic math in Python
- How to work with densities
- How to use uniform random numbers to generate random variables with any specified distribution
- How to find the density of a transformed random variable 

## Submission Instructions ##

Many assignments throughout the course will have a written portion and a code portion. Please follow the directions below to properly submit both portions.

### Written Portion ###
*  Scan all the pages into a PDF. You can use any scanner or a phone using applications such as CamScanner. Please **DO NOT** simply take pictures using your phone. 
* Please start a new page for each question. If you have already written multiple questions on the same page, you can crop the image in CamScanner or fold your page over (the old-fashioned way). This helps expedite grading.
* It is your responsibility to check that all the work on all the scanned pages is legible.
* If you used $\LaTeX$ to do the written portions, you do not need to do any scanning; you can just download the whole notebook as a PDF via LaTeX.

### Code Portion ###
* Save your notebook using `File > Save and Checkpoint`.
* Generate a PDF file using `File > Download As > PDF via LaTeX`. This might take a few seconds and will automatically download a PDF version of this notebook.
    * If you have issues, please post a follow-up on the general Lab 5B Ed thread.
    
### Submitting ###
* Combine the PDFs from the written and code portions into one PDF. [Here](https://smallpdf.com/merge-pdf) is a useful tool for doing so. 
* Submit the assignment to Lab 5B on Gradescope. 
* **Make sure to assign each page of your pdf to the correct question.**
* **It is your responsibility to verify that all of your work shows up in your final PDF submission.**

If you are having difficulties scanning, uploading, or submitting your work, please read the [Ed Thread](https://edstem.org/us/courses/35049/discussion/2398718) on this topic and post a follow-up on the general Lab 5B Ed thread.

## **We will not grade assignments which do not have pages selected for each question.** ##